   生成输入数据集 

In [12]:
from sgs import*

N_SAMPLE= 16
N_WELL=1
N_TASKS = 32
NUM_CONTRAL_POINT=10
PORO_VAR=0.004
PERM_VAR=8100
LEN_SCALE=[3200, 3200, 80]
x = np.arange(1900, 8200, 200)#32 6400
y = np.arange(1100, 7400, 200)#32 6400
z = np.arange(-2975, -2820, 10)#16 160


STEPS=[0,1,3,9,16,22,28,34,40,50]
T_STOP=28
INJ_RATE=15

GetAllTask(x,y,z,N_TASKS, N_SAMPLE,STEPS,T_STOP, INJ_RATE,PORO_VAR, PERM_VAR, LEN_SCALE, NUM_CONTRAL_POINT)

sim_0
T0 S0 PORO_COND mean:0.208189 , var:0.000621 , min:0.163349, max:0.246978 

T0 S0 PERM_COND mean:142.824327 , var:6512.178446 , min:33.201550, max:301.454000 

T0 S0 uPORO_VAL  mean:0.205253 , var:0.000142 , min:0.033516, max:0.368141 

T0 S0 uPERM_VAL  mean:122.269089 , var:1832.096930 , min:-255.716043, max:951.666035 

T0 S0 PORO_VAL  mean:0.205253 , var:0.000142 , min:0.163349, max:0.246978 

T0 S0 PERM_VAL  mean:122.269089 , var:1832.096930 , min:33.201550, max:301.454000 

sim_1
T0 S1 PORO_COND mean:0.208189 , var:0.000621 , min:0.163349, max:0.246978 

T0 S1 PERM_COND mean:142.824327 , var:6512.178446 , min:33.201550, max:301.454000 

T0 S1 uPORO_VAL  mean:0.202017 , var:0.000130 , min:0.043989, max:0.381570 

T0 S1 uPERM_VAL  mean:119.499587 , var:2069.343803 , min:-201.527394, max:894.184119 

T0 S1 PORO_VAL  mean:0.202017 , var:0.000130 , min:0.163349, max:0.246978 

T0 S1 PERM_VAL  mean:119.499587 , var:2069.343803 , min:33.201550, max:301.454000 

sim_2
T0 S2 PORO_CON

生成数据集标签

In [ ]:
import subprocess

path_comsol_server="D:\comsol\COMSOL62\Multiphysics\bin\win64\comsolmphserver.exe"####检查修改COMSOL的路径设置，符合你自己的PC设置
comsol_server_process = subprocess.Popen(path_comsol_server)
cmd_matlb_script="matlab -nosplash -nodesktop -r StarCOMSOL" ####检查修改StarCOMSOL中的路径设置，符合你自己的PC设置
os.system(cmd_matlb_script)
comsol_server_process.wait()

整理成数据集

In [ ]:
from cat_label import CatLabels
cols=np.linspace(3, 21, 10, dtype=np.int16)
catlabels = CatLabels("./", "new312","./dataSet",x,y,z,cols)
catlabels.DealAllTask()

进行训练

In [ ]:
from reptile_train import *

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

epochs_outer=1
epochs_inner=1
lr_outer=0.001
lr_inner=0.01

batch_size_outer = 1
batch_size_inner = 1

testSet=MateCo2Dataset("./dataSet", 3)
model = build_resunetplusplus()
model = model.double()
model = model.to(DEVICE)
train_loader = DataLoader(testSet, 1, True)
eval_loader = []
outter_loop(model, train_loader, epochs_outer, epochs_inner, eval_loader, lr_outer, batch_size_outer, lr_inner, batch_size_inner,device=DEVICE)